In [ ]:
from flask import Flask, request, jsonify, send_file
import cv2
from ultralytics import YOLO
import time
import os
from PIL import Image
import numpy as np

app = Flask(__name__)

# Load model and face detector once
model = YOLO('/Users/marcell/Documents/Work Space/Education/Semester 6/Cloud Computing/Cloud_Computing/python-image/Model.pt')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

confidence_threshold = 0.7

def check_overlap(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    if x_right <= x_left or y_bottom <= y_top:
        return False
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = w1 * h1
    box2_area = w2 * h2
    return intersection_area > 0.5 * min(box1_area, box2_area)

@app.route("/upload", methods=["POST"])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    # Read image into OpenCV format
    image_bytes = np.frombuffer(file.read(), np.uint8)
    frame = cv2.imdecode(image_bytes, cv2.IMREAD_COLOR)
    resized_frame = cv2.resize(frame, (854, 480))
    mirrored_frame = cv2.flip(resized_frame, 1)

    # YOLO inference
    
    results = model(mirrored_frame)
    annotated_frame = results[0].plot()

    # Face detection
    gray = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    smoker_detected = False
    for (x, y, w, h) in faces:
        for result in results[0].boxes:
            if result.cls == 0:
                confidence = result.conf.item()
                if confidence >= confidence_threshold:
                    cx, cy, cw, ch = result.xywh[0]
                    cigarette_box = (cx - cw / 2, cy - ch / 2, cw, ch)
                    if check_overlap((x, y, w, h), cigarette_box):
                        smoker_detected = True
                        cv2.rectangle(annotated_frame, (x - 20, y - 20), (x + w + 20, y + h + 20), (0, 0, 255), 4)
                        cv2.putText(annotated_frame, "Smoker Detected", (x, y - 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        break
        if not smoker_detected:
            cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), (255, 255, 0), 2)

    # Save the annotated image temporarily
    output_path = "output.jpg"
    cv2.imwrite(output_path, annotated_frame)

    return jsonify({
        "result": "Smoker Detected" if smoker_detected else "Not Detected",
        "image_url": "/result"
    })

@app.route("/result")
def serve_result_image():
    return send_file("output.jpg", mimetype='image/jpeg')

if __name__ == "__main__":
    app.run(debug=True)